In [1]:
# Ref: https://machinelearningmastery.com/multi-class-classification-tutorial-keras-deep-learning-library/

import pandas
from keras.models import Sequential
from keras.layers import Dense
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
import tensorflow as tf

2024-06-26 16:57:03.360646: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-26 16:57:04.268655: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


ModuleNotFoundError: No module named 'scikeras'

In [12]:
# load dataset
dataframe = pandas.read_csv("../images_data/mag5_adverserial_bins/mag5_adverserial_bins.csv", header=0 , names=["HIP", "bin0", "bin1", "bin2", "bin3", "bin4", "bin5", "bin6", "bin7", "bin8", "bin9"])
dataset = dataframe.values
X = dataset[:,1:11].astype(int)
Y = dataset[:,0].astype(str)
print("Input Size: ", len(X[0]))
print(X, '\n', Y)

Input Size:  10
[[ 3  1  4 ...  5  7  5]
 [ 3  1  4 ...  5  7  5]
 [ 3  1  4 ...  5  7  5]
 ...
 [ 3  4 14 ...  8  4  6]
 [ 3  4 14 ...  8  4  6]
 [ 3  4 14 ...  7  4  6]] 
 ['100027001' '100027001' '100027001' ... '99874001' '99874001' '99874001']


In [13]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = tf.keras.utils.to_categorical(encoded_Y)
print("Output Size:", len(dummy_y[0]))

Output Size: 1608


In [14]:
# define baseline model
def baseline_model():
 # create model
 model = Sequential()
 model.add(Dense(64, input_dim=10, activation='relu'))
 model.add(Dense(32, activation='relu'))
 model.add(Dense(1608, activation='softmax'))
 # Compile model
 model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
 return model

In [15]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=10, batch_size=5, verbose=0)

In [16]:
kfold = KFold(n_splits=10, shuffle=True)

In [17]:
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

c:\Users\keepi\AppData\Local\Programs\Python\Python312\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\keepi\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\keepi\AppData\Local\Programs\Python\Python312\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\keepi\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core

Baseline: 95.53% (0.56%)
